Curve: $$y^2= x^3 + ax + b$$ \
$$where a = 0 & b = 7$$ \
The specific curve is the secp256K1
$$y^2= x^3 + 7$$

https://www.bitaddress.org/bitaddress.org-v3.3.0-SHA256-dec17c07685e1870960903d8f58090475b25af946fe95a734f88408cef4aa194.html

In [ ]:
prime=2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1

In [7]:
# base10
prime 

115792089237316195423570985008687907853269984665640564039457584007908834671663

In [8]:
hex(prime)

'0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f'

G is the generator point and G is defined by a & b \
n is the points in the field \
h is the co-factor

In [51]:
n=0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

In [20]:
a_curve = 0
b_curve = 7

In [13]:
h = 1

when there is an x point on the curve, there is an y point \
They are the same point that flipped over x-axis. \
So if you know x on any point of the curve, you know the value of y.

In [12]:
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
y = 0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8        

In [159]:
hex(Gx)

'0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798'

In [161]:
x

55066263022277343669578718895168534326250603453777594175500187360389116729240

In [162]:
hex(Gy)

'0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8'

In [163]:
y

32670510020758816978083085130507043184471273380659243275938904335757337482424

In [164]:
G = (Gx, Gy)

In [165]:
G

(55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)

In [149]:
privateKey = 0xA0DC65FFCA799873CBEA0AC274015B9526505DAAAED385155425F7337704883E

In [166]:
privateKey = 0x931609513E57AF7CB252DD5C22E76D21A035A145C60AF8E9C7856CAB823C8971

In [155]:
privateKey

66528923683203518504868520852346299273540010994337822668999060952421429447025

In [105]:
privateKeyBin = str(bin(privateKey))[2:]
privateKeyBin

'1010000011011100011001011111111111001010011110011001100001110011110010111110101000001010110000100111010000000001010110111001010100100110010100000101110110101010101011101101001110000101000101010101010000100101111101110011001101110111000001001000100000111110'

In [167]:
str(bin(privateKey))

'0b1001001100010110000010010101000100111110010101111010111101111100101100100101001011011101010111000010001011100111011011010010000110100000001101011010000101000101110001100000101011111000111010011100011110000101011011001010101110000010001111001000100101110001'

Uncompressed public key is the result of private key multiply generation point
$$publicKey = privateKey * G$$

In [156]:
from os import urandom
import hashlib

class SECP256k1:
    def __init__(self,
        x=0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
        y=0x483ADA7726A3C4655DA4FBFC0E1108A8FD17B448A68554199C47D08FFB10D4B8,
        p=2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1):
        
        self.x = x
        self.y = y
        self.p = p

    def __add__(self, other):
        return self.__radd__(other)

    def __mul__(self, other):
        return self.__rmul__(other)

    def __rmul__(self, other):
        n = self
        q = None

        for i in range(256):
            if other & (1 << i):
                q = q + n
            n = n + n
        return q

    def __radd__(self, other):
        if other is None:
            return self
        x1 = other.x
        y1 = other.y
        x2 = self.x
        y2 = self.y
        p = self.p

        if self == other:
            l = pow(2 * y2 % p, p-2, p) * (3 * x2 * x2) % p
        else:
            l = pow(x1 - x2, p-2, p) * (y1 - y2) % p

        newX = (l ** 2 - x2 - x1) % p
        newY = (l * x2 - l * newX - y2) % p

        return SECP256k1(newX, newY)

    def toBytes(self):
        x = self.x.to_bytes(32, "big")
        y = self.y.to_bytes(32, "big")
        return b"\x04" + x + y

    
def sha256(data):
    digest = hashlib.sha256()
    digest.update(data)
    return digest.digest()


def ripemd160(x):
    d = hashlib.new("ripemd160")
    d.update(x)
    return d.digest()


def base58(data):
    alphabet = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"

    if data[0] == 0:
        return "1" + base58(data[1:])

    x = sum([v * (256 ** i) for i, v in enumerate(data[::-1])])
    ret = ""
    while x > 0:
        ret = alphabet[x % 58] + ret
        x = x // 58

    return ret

def getPublicKey(privkey):
    secp256k1 = SECP256k1()
#     pk = int.from_bytes(privkey, "big")
    pk = privateKey
    hash160 = ripemd160(sha256((secp256k1 * pk).toBytes()))
    address = b"\x00" + hash160

    address = base58(address + sha256(sha256(address))[:4])
    return address


def getWif(privkey):
    wif = b"\x80" + privkey
    wif = base58(wif + sha256(sha256(wif))[:4])
    return wif


if __name__ == "__main__":
    randomBytes = urandom(32)
    print("Address: " + getPublicKey(randomBytes))
#     print("Privkey: " + getWif(randomBytes))




Address: 1PL91Bp92ESE2gbSSx7Jf1H3vZno6zVoxr


In [147]:
ans = multi(privateKey, G)

In [144]:
sec = int.from_bytes(randomBytes,byteorder='big')
sec

113222265672974413393418523735460271466253646155597497947161490565543258819931

In [141]:
int.from_bytes(randomBytes, "big")

113222265672974413393418523735460271466253646155597497947161490565543258819931

In [108]:
hex(ans)

'0xa0dc65ffca799873cbea0ac274015b9526505daaaed385155425f7337704883e'

In [116]:
type(ans)

int

In [109]:
EcDivision((2* G[0]), prime)

9007199254740992.0

In [157]:
ans = multi(privateKey, G)

In [158]:
ans

66528923683203518504868520852346299273540010994337822668999060952421429447025

In [113]:
hex(ans)

'0xa0dc65ffca799873cbea0ac274015b9526505daaaed385155425f7337704883e'

Ex:
https://walletgenerator.net/
https://kimbatt.github.io/btc-address-generator/

Refs:
https://en.bitcoin.it/wiki/Invoice_address
https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm
https://github.com/opsxcq/blog/blob/master/static/src/bitcoin-address-generator.py

